Code to genearte an NLP Based search engine on specific documents 

Data is from Kaggle on products and the code combined several fields as descriptor for the product

Then the NLP process ahave a search option to produce the closest matches. 

In [1]:
import kagglehub
import os
import pandas as pd 
# Download latest version of this dataset: https://www.kaggle.com/datasets/sujaykapadnis/products-datasets
path = kagglehub.dataset_download("sujaykapadnis/products-datasets")
documents = {}
print("Path to dataset files:", path)
try: 
    prod_data = pd.read_csv(path+os.listdir(path)[0])
    for index, row in prod_data.iterrows():
        documents[row['S.No']] = row['Product Name']+" "+row['Brand Desc']+" "+row['Product Size']+" "+row['Category']
except:
    path = '/Users/jorgepinzon/.cache/kagglehub/datasets/sujaykapadnis/products-datasets/versions/1/'
    prod_data = pd.read_csv(path+os.listdir(path)[0])
    for index, row in prod_data.iterrows():
        documents[row['S.No']] = row['Product Name']+" "+row['Brand Desc']+" "+row['Product Size']+" "+row['Category']
len(documents)

Path to dataset files: /Users/jorgepinzon/.cache/kagglehub/datasets/sujaykapadnis/products-datasets/versions/1


4566

In [2]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/jorgepinzon/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/jorgepinzon/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
# WITH OUT CHECKING FOR SYNONIMS 
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

# Download NLTK stopwords (run once)



def preprocess(text):
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text.lower())
    tokens = text.split()
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]
    return ' '.join(tokens)

# # Sample documents
# documents = {
#     "doc1": "Python is a programming language",
#     "doc2": "Python is used for web development and data science",
#     "doc3": "Natural Language Processing powers search engines",
# }

# Preprocess docs
processed_docs = [preprocess(doc) for doc in documents.values()]

# TF-IDF Vectorization
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(processed_docs)

# Search function
def search(query):
    processed_query = preprocess(query)
    query_vec = vectorizer.transform([processed_query])
    cos_sim = cosine_similarity(query_vec, tfidf_matrix)
    scores = cos_sim.flatten()
    ranked_docs = sorted(zip(documents.keys(), scores), key=lambda x: x[1], reverse=True)
    return ranked_docs, [doc for doc, score in ranked_docs if score > 0]



In [4]:
ranked_docs, results = search("cozzy shirt")
for key in results: 
    print(f'''{key}:\t {documents[key]}''') 

111:	 DRW110 - Westernwear-Women solid shirt collar regular fit womens casual shirt - olive Size:X-Large Westernwear-Women
32:	 DRW31 - Westernwear-Women stripes shirt collar rayon womens casual shirt - off white Size:Large,Medium,Small,XX-Large Westernwear-Women
93:	 DRW92 - Westernwear-Women checkered shirt collar polyester cotton womens casual shirt - blue Size:Large,Medium,Small,X-Large Westernwear-Women
4098:	 CHARCHIT1 - Westernwear-Women checks cotton shirt collar boys shirt - white Size:13-14Y Westernwear-Women
486:	 ALLEN SOLLY20 - Westernwear-Women solid polyester v neck womens shirt - pink Size:Large,Medium,Small,X-Large,X-Small,XX-Large Westernwear-Women
510:	 ALLEN SOLLY44 - Westernwear-Women solid polyester v neck womens shirt - off white Size:Large,Medium,Small,X-Large,X-Small,XX-Large Westernwear-Women
66:	 DRW65 - Westernwear-Women womens printed casual shirt - black Size:X-Small Westernwear-Women
512:	 ALLEN SOLLY46 - Westernwear-Women solid cotton v neck womens shirt

In [5]:
import re
from nltk.corpus import stopwords, wordnet
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from symspellpy import SymSpell
import pkg_resources

# Initialize SymSpell for spell-check
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
dictionary_path = pkg_resources.resource_filename("symspellpy", "frequency_dictionary_en_82_765.txt")
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

# Text preprocessing
def syn_preprocess(text):
    text = re.sub(r'[^a-zA-Z0-9\s]', '', text.lower())
    tokens = text.split()
    stop_words = set(stopwords.words('english'))
    tokens = [word for word in tokens if word not in stop_words]
    stemmer = PorterStemmer()
    tokens = [stemmer.stem(word) for word in tokens]
    return ' '.join(tokens)

# Spell-check
def spell_check(query):
    suggestions = sym_spell.lookup_compound(query, max_edit_distance=2)
    return suggestions[0].term

# Synonyms
def get_synonyms(word):
    synonyms = set()
    for syn in wordnet.synsets(word):
        for lemma in syn.lemmas():
            synonyms.add(lemma.name())
    return list(synonyms)

def expand_query_with_synonyms(query):
    tokens = query.split()
    expanded_query = []
    for token in tokens:
        expanded_query.append(token)
        synonyms = get_synonyms(token)
        expanded_query.extend(synonyms[:2])  # Add top 2 synonyms
    return ' '.join(expanded_query)

# Preprocess docs
processed_docs = [syn_preprocess(doc) for doc in documents.values()]

# TF-IDF Vectorization
vectorizer = TfidfVectorizer()
tfidf_matrix = vectorizer.fit_transform(processed_docs)

# Search function
def syn_search(query):
    # Spell-check
    corrected_query = spell_check(query)
    print(f"Corrected Query: {corrected_query}")
    
    # Expand with synonyms
    expanded_query = expand_query_with_synonyms(corrected_query)
    print(f"Expanded Query: {expanded_query}")
    
    # Preprocess query
    processed_query = syn_preprocess(expanded_query)
    
    # Vectorize and compare
    query_vec = vectorizer.transform([processed_query])
    cos_sim = cosine_similarity(query_vec, tfidf_matrix)
    scores = cos_sim.flatten()
    
    # Rank documents
    ranked_docs = sorted(zip(documents.keys(), scores), key=lambda x: x[1], reverse=True)
    return ranked_docs, [doc for doc, score in ranked_docs if score >0]

In [6]:
scores, results = syn_search("I need a warm t-shirt for a boy, not a perfume, not too big but also not small")
print(len(results))
for key in results: 
    print(f'''{key}:\t {documents[key]}''') 

Corrected Query: i need a warm to shirt for a boy not a perfume not too big but also not small
Expanded Query: i atomic_number_53 I need penury postulate a a angstrom warm warm_up affectionate to shirt shirt for a a angstrom boy male_child boy not not non a a angstrom perfume aromatise perfume not not non too likewise to_a_fault big boastful vauntingly but merely but also likewise too not not non small low humble
245
4098:	 CHARCHIT1 - Westernwear-Women checks cotton shirt collar boys shirt - white Size:13-14Y Westernwear-Women
4332:	 CLOVIA109 - Lingerie&Nightwear-Women womens cotton rich non padded non wired t shirt bra - red Size:32-C,32-D,34-B,36-B,38-B Lingerie&Nightwear-Women
4274:	 CLOVIA51 - Lingerie&Nightwear-Women womens cotton non wired non padded full cup t shirt bra blue - blue Size:32-C,34-B,36-B Lingerie&Nightwear-Women
4303:	 CLOVIA80 - Lingerie&Nightwear-Women womens non padded non wired solid t-shirt bra - white Size:32-B,32-C Lingerie&Nightwear-Women
1065:	 AMANTE184

In [18]:
for doc, score in scores:
    if score > 0.1:
        print(f"Document: {doc}, Score: {score:.2f}")

Document: 4098, Score: 0.33
Document: 4332, Score: 0.29
Document: 4274, Score: 0.27
Document: 4303, Score: 0.24
Document: 1065, Score: 0.24
Document: 887, Score: 0.24
Document: 4309, Score: 0.23
Document: 4360, Score: 0.23
Document: 4292, Score: 0.23
Document: 941, Score: 0.23
Document: 4363, Score: 0.23
Document: 917, Score: 0.23
Document: 1013, Score: 0.23
Document: 1062, Score: 0.23
Document: 962, Score: 0.23
Document: 969, Score: 0.23
Document: 930, Score: 0.22
Document: 971, Score: 0.22
Document: 899, Score: 0.22
Document: 986, Score: 0.22
Document: 1056, Score: 0.22
Document: 4300, Score: 0.22
Document: 1049, Score: 0.22
Document: 1012, Score: 0.22
Document: 4343, Score: 0.22
Document: 883, Score: 0.22
Document: 4297, Score: 0.22
Document: 943, Score: 0.22
Document: 1058, Score: 0.22
Document: 955, Score: 0.22
Document: 905, Score: 0.21
Document: 945, Score: 0.21
Document: 4285, Score: 0.21
Document: 1064, Score: 0.21
Document: 895, Score: 0.20
Document: 1036, Score: 0.20
Documen

In [8]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords, wordnet
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from symspellpy import SymSpell
import pkg_resources
import re
from rank_bm25 import BM25Okapi


# Initialize tools
lemmatizer = WordNetLemmatizer()
stop_words = set(stopwords.words('english'))
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)
dictionary_path = pkg_resources.resource_filename("symspellpy", "frequency_dictionary_en_82_765.txt")
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

# Preprocessing
def preprocess(text):
    text = re.sub(r'[^a-zA-Z\s]', '', text.lower())
    tokens = text.split()
    tokens = [lemmatizer.lemmatize(word) for word in tokens if word not in stop_words]
    return ' '.join(tokens)

# Spell-check
def spell_check(query):
    suggestions = sym_spell.lookup_compound(query, max_edit_distance=2)
    return suggestions[0].term

# Query expansion
def expand_query_with_synonyms(query):
    tokens = query.split()
    expanded_query = []
    for token in tokens:
        expanded_query.append(token)
        synonyms = set()
        for syn in wordnet.synsets(token):
            for lemma in syn.lemmas():
                synonyms.add(lemma.name())
        expanded_query.extend(list(synonyms)[:2])
    return ' '.join(expanded_query)
# Preprocess documents
processed_docs = [preprocess(doc) for doc in documents.values()]

# TF-IDF Vectorization with n-grams
vectorizer = TfidfVectorizer(ngram_range=(1, 2))
tfidf_matrix = vectorizer.fit_transform(processed_docs)

#BM25
# Tokenize documents
tokenized_docs = [doc.split() for doc in processed_docs]
# Initialize BM25
bm25 = BM25Okapi(tokenized_docs)
# BERT: 
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity
import torch
# Load pre-trained Sentence-BERT model
model = SentenceTransformer('all-MiniLM-L6-v2')  # Lightweight and fast model
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
# Encode documents in batches
def encode_in_batches(texts, model, batch_size=32):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch = texts[i:i + batch_size]
        batch_embeddings = model.encode(batch, device=device)
        embeddings.extend(batch_embeddings)
    return embeddings

doc_embeddings = encode_in_batches(processed_docs, model, batch_size=32)

# Search function
def impr_search(query):
    # Spell-check
    corrected_query = spell_check(query)
    print(f"Corrected Query: {corrected_query}")
    
    # Expand with synonyms
    expanded_query = expand_query_with_synonyms(corrected_query)
    print(f"Expanded Query: {expanded_query}")
    
    # Preprocess query
    processed_query = preprocess(expanded_query)
    
    # Vectorize and compare
    query_vec = vectorizer.transform([processed_query])
    cos_sim = cosine_similarity(query_vec, tfidf_matrix)
    scores = cos_sim.flatten()
    
    # Rank documents
    ranked_docs = sorted(zip(documents.keys(), scores), key=lambda x: x[1], reverse=True)
    return ranked_docs, [doc for doc, score in ranked_docs if score > 0]

def bmk_search(query):
    # Preprocess query
    processed_query = preprocess(query)
    query_tokens = processed_query.split()
    
    # Get BM25 scores
    scores = bm25.get_scores(query_tokens)
    
    # Rank documents
    ranked_docs = sorted(zip(documents.keys(), scores), key=lambda x: x[1], reverse=True)
    return ranked_docs, [doc for doc, score in ranked_docs if score > 0]

def bert_search(query):
    # Preprocess query
    processed_query = preprocess(query)
    
    # Encode query
    query_embedding = model.encode([processed_query], device=device)
    
    # Compute cosine similarity
    scores = cosine_similarity(query_embedding, doc_embeddings).flatten()
    
    # Rank documents
    ranked_docs = sorted(zip(documents.keys(), scores), key=lambda x: x[1], reverse=True)
    return ranked_docs, [doc for doc, score in ranked_docs if score > 0]

/opt/anaconda3/envs/p312/lib/python3.12/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


cpu


In [14]:
sentence = '''Can you suggest a nice cologne for my son? '''

In [15]:
ranked_docs, results = impr_search(sentence)
print(len(results))
for key in results: 
    print(f'''{key}:\t {documents[key]}''') 

Corrected Query: can you suggest a nice cologne for my son
Expanded Query: can bathroom displace you suggest intimate paint_a_picture a a angstrom nice courteous gracious cologne eau_de_cologne Cologne for my son boy Son
9
1:	 Cologne Fragrance ekw eau de cologne 400 ml Small Fragrance-Women
2154:	 ASPEN1 - Fragrance-Women mens eau de cologne 118 ml Nan Fragrance-Women
4236:	 CLOVIA13 - Lingerie&Nightwear-Women womens solid boy shorts briefs - black Size:Medium,Small,X-Large,XX-Large Lingerie&Nightwear-Women
977:	 AMANTE96 - Lingerie&Nightwear-Women womens solid boy leg briefs - royal blue Size:Large,Small,X-Large Lingerie&Nightwear-Women
1065:	 AMANTE184 - Lingerie&Nightwear-Women printed cotton low rise womens boy shorts - grape Size:Large,Medium,Small,X-Large Lingerie&Nightwear-Women
1017:	 AMANTE136 - Lingerie&Nightwear-Women womens solid seamless boy leg briefs - natural Size:Large,Small Lingerie&Nightwear-Women
907:	 AMANTE26 - Lingerie&Nightwear-Women cotton blend intimatewear w

In [16]:
ranked, results = bmk_search(sentence)
print(len(results))
for key in results: 
    print(f'''{key}:\t {documents[key]}''') 

2
1:	 Cologne Fragrance ekw eau de cologne 400 ml Small Fragrance-Women
2154:	 ASPEN1 - Fragrance-Women mens eau de cologne 118 ml Nan Fragrance-Women


In [17]:
ranked, results = bert_search(sentence)
print(len(results))
for key in results: 
    print(f'''{key}:\t {documents[key]}''') 

4566
1:	 Cologne Fragrance ekw eau de cologne 400 ml Small Fragrance-Women
2154:	 ASPEN1 - Fragrance-Women mens eau de cologne 118 ml Nan Fragrance-Women
422:	 AJMAL5 - Fragrance-Women mens blu classic pour homme deodorant 200 ml Nan Fragrance-Women
421:	 AJMAL4 - Fragrance-Women mens blu classic pour homme deodorant 200 ml Nan Fragrance-Women
3551:	 CAROLINA HERRERA10 - Fragrance-Women very good girl eau de parfum 80 ml with body lotion 100 ml and miniature 7 ml Nan Fragrance-Women
434:	 AJMAL17 - Fragrance-Women unisex distraction deodorant spicy fragrance 200ml Nan Fragrance-Women
3561:	 CAROLINA HERRERA20 - Fragrance-Women very good girl eau de parfum 50 ml with body lotion 100 ml Nan Fragrance-Women
429:	 AJMAL12 - Fragrance-Women mens carbon classic pour homme deodorant 200 ml Nan Fragrance-Women
3402:	 BURBERRY9 - Fragrance-Women london eau de parfum - 100 ml Nan Fragrance-Women
3547:	 CAROLINA HERRERA6 - Fragrance-Women very good girl 80 ml with liquid lipstick Nan Fragrance-Wo